In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv("cleaned_data.csv")
df.head()

,Unnamed: 0,AFE101,AFE103,BIO101,CHM101,CSC101,GST111,GST113,GST115,MAT101,...,CSC308,CSC310,CSC312,CSC314,GST212.1,MAT204.1,PHY102.2,PHY108.2,PHY202.1,300L_2ND_CGPA
0,0,5.0,0.0,0.0,2.0,3.0,2.0,2.0,4.0,3.0,...,5.0,5.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,3.26
1,1,3.0,0.0,0.0,2.0,0.0,0.0,4.0,2.0,2.0,...,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.36
2,2,3.0,4.0,0.0,2.0,3.0,2.0,0.0,3.0,2.0,...,4.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.26
3,3,5.0,4.0,0.0,4.0,5.0,4.0,4.0,5.0,2.0,...,5.0,5.0,4.0,5.0,3.0,0.0,0.0,0.0,0.0,3.95
4,4,0.0,5.0,0.0,2.0,2.0,3.0,3.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.60


In [3]:
input = df.drop(["Unnamed: 0","300L_2ND_CGPA"], axis=1)
target = df[["300L_2ND_CGPA"]]


In [4]:
target.head()

,300L_2ND_CGPA
0,3.26
1,2.36
2,2.26
3,3.95
4,2.60


In [5]:
input.head()

,AFE101,AFE103,BIO101,CHM101,CSC101,GST111,GST113,GST115,MAT101,PHY101,...,CSC306,CSC308,CSC310,CSC312,CSC314,GST212.1,MAT204.1,PHY102.2,PHY108.2,PHY202.1
0,5.0,0.0,0.0,2.0,3.0,2.0,2.0,4.0,3.0,2.0,...,3.0,5.0,5.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
1,3.0,0.0,0.0,2.0,0.0,0.0,4.0,2.0,2.0,2.0,...,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,4.0,0.0,2.0,3.0,2.0,0.0,3.0,2.0,2.0,...,2.0,4.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,5.0,4.0,0.0,4.0,5.0,4.0,4.0,5.0,2.0,5.0,...,5.0,5.0,5.0,4.0,5.0,3.0,0.0,0.0,0.0,0.0
4,0.0,5.0,0.0,2.0,2.0,3.0,3.0,4.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Splitting file into different columns based on our model prediction requirement
model1_columns = []
model2_columns = []
model3_columns = []
# FACTORS IN ALL CARRYOVERS IF ANY IN MODEL 4
model4_columns = []
for item in [input]:
 for i in item:
        if len(i) == 6 and i[3] == '1':
            model1_columns.append(i)
        elif len(i) == 6 and i[3] == '2':
            model2_columns.append(i)
        elif len(i) == 6 and i[3] == '3':
            model3_columns.append(i)
        else:
            model4_columns.append(i)

model2_columns = model1_columns + model2_columns
model3_columns = model2_columns + model3_columns
model4_columns = model3_columns + model4_columns

In [7]:
# Input data
labels = target.values
features = df[model3_columns].values

In [8]:
print(model3_columns)

['AFE101', 'AFE103', 'BIO101', 'CHM101', 'CSC101', 'GST111', 'GST113', 'GST115', 'MAT101', 'PHY101', 'PHY103', 'PHY107', 'STA111', 'AFE124', 'AFE126', 'AFE128', 'AFE130', 'AFE132', 'CSC102', 'CSC104', 'GST114', 'GST122', 'MAT102', 'MAT104', 'PHY102', 'PHY108', 'STA132', 'AFE203', 'CSC201', 'CSC203', 'CSC205', 'CSC207', 'CSC211', 'MAT201', 'MAT205', 'STA211', 'AFE202', 'CSC202', 'CSC204', 'CSC206', 'CSC208', 'GST212', 'MAT204', 'PHY202', 'CSC299', 'CSC301', 'CSC303', 'CSC305', 'CSC307', 'CSC309', 'CSC311', 'CSC315', 'GST301', 'CSC302', 'CSC304', 'CSC306', 'CSC308', 'CSC310', 'CSC312', 'CSC314']


In [9]:
len(model3_columns)

60

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

In [32]:
model_X = Sequential([
    layers.Dense(64, activation='relu', input_dim=(60)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
# Compilation
model_X.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [13]:
# Model Training
model_X.fit(X_train, y_train, epochs=290, batch_size=32, verbose=0)

In [14]:
# Model Evaluation
loss, MAE = model_X.evaluate(X_test, y_test)
print(f"Test Mean Absolute Error: {MAE}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.1278 - mae: 0.2877
Test Mean Absolute Error: 0.2877046763896942


In [15]:
# Predicting
y_pred = model_X.predict(X_test).flatten()  # Flatten is used to make sure y_pred is the same shape as y_test

# Evaluating the model
test_loss, test_mae = model_X.evaluate(X_test, y_test, verbose=0)

# Calculating R-squared
r_squared = r2_score(y_test, y_pred)

# Displaying results
print(f"Test Mean Absolute Error: {test_mae}")
print(f"R-squared: {r_squared}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Test Mean Absolute Error: 0.2877046763896942
R-squared: 0.7440125759981748


In [16]:
model_X.save('DURU.keras') 

In [17]:
from keras.models import load_model
model = load_model('DURU.keras')

In [19]:
# Flatten the arrays
y_test_flat = y_test.flatten()
y_pred_flat = y_pred.flatten()

# Create a DataFrame to compare actual and predicted labels
df_compare = pd.DataFrame({"Actual": y_test_flat, "Predict": y_pred_flat})

# Print or visualize the comparison
df_compare

,Actual,Predict
0,2.29,1.989442
1,3.13,3.600403
2,2.06,1.240885
3,3.68,3.506891
4,4.42,4.514519
5,3.93,3.786335
6,3.06,2.648920
7,2.26,1.907313
8,3.60,3.335051
9,3.16,2.659874


In [31]:
input_data = [ 5, 5, 5, 5, 2, 5, 3, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5,5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5,]

# Reshape input data to fit the model input shape (1, number of features)
input_array = np.array([input_data])  # Assuming the model expects 20 features

# Predict using the loaded model
prediction = model.predict(input_array).flatten()
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
[4.6945295]


## Deep Learning (RNN) ❌

In [27]:
RNN_model = Sequential([
    layers.Embedding(input_dim=6, output_dim=50, input_shape=(117,)),
    layers.SimpleRNN(units=50, return_sequences=True),
    layers.SimpleRNN(units=50),
    layers.Dense(1, activation='sigmoid')
])

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [28]:
RNN_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
RNN_model.fit(X_train, y_train, epochs=290, batch_size=32, verbose=0)

# Evaluating the model
y_pred = RNN_model.predict(X_test)
test_loss, test_mae = RNN_model.evaluate(X_test, y_test)

print(f"Test Mean Absolute Error: {test_mae}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - loss: 5.7756 - mae: 2.2970
Test Mean Absolute Error: 2.2970213890075684


In [29]:
# Flatten the arrays
y_test_flat = y_test.flatten()
y_pred_flat = y_pred.flatten()

# Create a DataFrame to compare actual and predicted labels
df_compare = pd.DataFrame({"Actual": y_test_flat, "Predict": y_pred_flat})

# Print or visualize the comparison
df_compare

,Actual,Predict
0,2.29,0.999935
1,3.13,0.999943
2,2.06,0.994631
3,3.68,0.999941
4,4.42,0.999946
5,3.93,0.999881
6,3.06,0.999889
7,2.26,0.999903
8,3.60,0.999872
9,3.16,0.999914
